 \\[
 \begin{aligned}
 \frac{dS}{dt} &= -\frac{\beta SI}{N} \\
 \frac{dI}{dt} &= \frac{\beta SI}{N} - \gamma I\\
 \frac{dR}{dt} &= \gamma I 
 \end{aligned}
 \\]


In [1]:
import numpy as np
import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots

In [249]:
class SIR_Model:
    def __init__(self, beta, gamma, dt=1):
        self.beta = beta*dt
        self.gamma = gamma*dt
        self.R0 = beta / gamma
        
    def __str__(self):
        return "β=" + str(self.beta) + " γ=" + str(self.gamma)
    
    def inhabit(self, S0, I0, R0):
        self.S0 = S0
        self.I0 = I0
        self.R0 = R0
        self.N = S0 + I0 + R0
        
    def epidemic(self, T):
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        S[0] = self.S0
        I[0] = self.I0
        R[0] = self.R0

        for t in range(T):
            a, b = self.beta*S[t]*I[t]/self.N, self.gamma*I[t]
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b
            R[t+1] = R[t] + b
            
        self.S = S
        self.I = I
        self.R = R
        self.T = T
        
    def plot(self):
        fig = go.Figure()
#        fig.update_yaxes(range=(0, self.N))
        fig.update_layout(
            margin=dict(b=0, l=0, r=0, t=50)
        )
        T = self.T
        fig.add_scatter(x=np.arange(T+1), y=self.S.astype(int), name="Susceptible", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.I.astype(int), name="Infectious", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.R.astype(int), name="Removed", hovertemplate="%{y}")
        return fig
    
    def sample(self, time, export, f):
        for t, e in zip(time, export):
            yield f(e, self.I[t]/self.N)

In [250]:
def binomial(trial, density):
    return np.random.binomial(trial, density)

In [251]:
def poisson(trial, density):
    return np.random.poisson(trial*density)

In [252]:
a = SIR_Model(3, 1, 0.01)

In [253]:
a.inhabit(1e7-1e4, 1e4, 0)

In [254]:
a.epidemic(1000)

In [255]:
np.random.seed(0)
b_design = np.arange(100, 1000, 100)
export = 10*np.ones_like(b_design)
positive = np.array(list(a.sample(b_design, export, poisson)))

fig = a.plot()
fig.add_scatter(x=b_design, y=positive/export*a.N, mode="markers", name="Guessed", hovertemplate="%{y}")

In [248]:
positive

array([0, 1, 3, 7, 0, 0, 2, 1, 0])